# Контекстно-свободные грамматики  
Курс NLP, семинар 11.  
Осень 2017.

Подпись: (введите сюда свои ФИО + отделение + курс)

![](http://i.imgur.com/XLREeKo.png)

## Немного теории

*Формальная грамматика* (определение с лекции):
 - конечное множество нетерминалов $P$;
 - конечное множество терминалов $\Sigma$;
 - конечное множество продукций $P$, каждая из которых имеет вид
 
$$(\Sigma \cup N)^∗ N (\Sigma \cup N)^∗ \rightarrow (\Sigma \cup N)^∗$$
 - начальный нетерминал $S$.
 
*Контекстно-свободная грамматика* имеет правила вида $A \rightarrow \beta, \beta \in (N \cup \Sigma)^*$.


Контекстно-свободная является грамматикой в *нормальной форме Хомского*, если содержит только правила вида: 
 - $A \rightarrow B C$
 - $A \rightarrow a$
 - $S \rightarrow \varepsilon$

где $a$ $-$ терминал, $A, B, C$ $-$ нетерминалы, $S$ $-$ начальный нетерминал не содержащийся в правых частях правил, $\varepsilon$ $-$ пустая строка.

Как привести грамматику к нормальной форме Хомского можно прочитать [здесь](http://math.stackexchange.com/questions/296243/converting-to-chomsky-normal-form) или [здесь](http://neerc.ifmo.ru/wiki/index.php?title=Нормальная_форма_Хомского#.D0.9F.D1.80.D0.B8.D0.B2.D0.B5.D0.B4.D0.B5.D0.BD.D0.B8.D0.B5_.D0.B3.D1.80.D0.B0.D0.BC.D0.BC.D0.B0.D1.82.D0.B8.D0.BA.D0.B8_.D0.BA_.D0.BD.D0.BE.D1.80.D0.BC.D0.B0.D0.BB.D1.8C.D0.BD.D0.BE.D0.B9_.D1.84.D0.BE.D1.80.D0.BC.D0.B5_.D0.A5.D0.BE.D0.BC.D1.81.D0.BA.D0.BE.D0.B3.D0.BE).

## 1. Chomsky Normal Form 
Приведите грамматику к нормальной форме Хомского (и выпишите итоговую <font color='red'>внутри ipython ноутбука</font>):

- $S \rightarrow NP ~~ VP$
- $NP \rightarrow DET ~~ ADJ ~~ N \mid NN$
- $VP \rightarrow V ~~ NP \mid VP ~~ PP \mid V$
- $DET \rightarrow a$
- $ADJ \rightarrow tasty \mid ADV ~~ ADJ \mid \epsilon$
- $ADV \rightarrow very$
- $N \rightarrow fish \mid fork \mid dog \mid boy$
- $NN \rightarrow Mary \mid John$
- $V \rightarrow eats$
- $PP \rightarrow P ~~ NP \mid \epsilon$
- $P \rightarrow with$


## 2. CYK Parser

Реализуйте синтаксический парсер, работающий по методу [**CYK**](https://ru.wikipedia.org/wiki/%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC_%D0%9A%D0%BE%D0%BA%D0%B0_%E2%80%94_%D0%AF%D0%BD%D0%B3%D0%B5%D1%80%D0%B0_%E2%80%94_%D0%9A%D0%B0%D1%81%D0%B0%D0%BC%D0%B8) и выполняющий функцию проверки принадлежности строки грамматике.

In [41]:
from itertools import product

class CYKParser:
    def __init__(self, rules):
        self.rules = rules
        
    def get_non_terminals_dict(self):
        non_terminals_dict = {}
        i = 0
        for start in list(sorted(self.rules.keys())):
            if start not in non_terminals_dict:
                non_terminals_dict[start] = i
                i += 1
            for end in self.rules[start]:
                parts = end.split(' ')
                if len(parts) == 2:
                    for part in parts:
                        if part not in non_terminals_dict:
                            non_terminals_dict[part] = i
                            i += 1
        return non_terminals_dict
    
    def parse(self, in_tokens):
        ntd = self.get_non_terminals_dict()
        p = []
        for i in range(len(in_tokens)):
            p.append([])
            for j in range(len(in_tokens)):
                p[i].append([])
                for k in range(len(ntd)):
                    p[i][j].append(False)
        for i in range(len(in_tokens)):
            word = in_tokens[i]
            for start in self.rules.keys():
                for end in self.rules[start]:
                    if len(end.split(' ')) == 1:
                        if end == word:
                            p[0][i][ntd[start]] = True
        for i in range(2, len(in_tokens) + 1):
            for j in range(1, len(in_tokens) - i + 2):
                for k in range(1, i):
                    for start in self.rules.keys():
                        for end in self.rules[start]:
                            parts = end.split(' ')
                            if len(parts) == 2:
                                if p[k-1][j-1][ntd[parts[0]]] and p[i - k - 1][j + k - 1][ntd[parts[1]]]:
                                    p[i-1][j-1][ntd[start]] = True
        return p[len(in_tokens) - 1][0][ntd['S']]

Введите сюда полученную грамматику из первого пункта.

In [42]:
RULES = {
    'S': ['NP VP'],
    ### YOUR GRAMMAR IN CNF HERE
    'NP': ['Mary', 'John', 'DET T_NP', 'DET N', ],
    'T_NP': ['ADJ N', 'ADV N'],
    'VP': ['V NP', 'VP PP', 'VP V', 'VP VP'],
    'DET': ['a'],
    'ADJ': ['tasty', 'ADV ADJ', 'ADV ADV'],
    'ADV': ['very'],
    'N': ['fish', 'fork', 'dog', 'boy'],
    'NN': [],
    'V': ['eats'],
    'PP': ['P NP'],
    'P': ['with'],
    ### END OF YOUR GRAMMAR
}

In [43]:
kek = CYKParser(RULES)
#print(kek.get)
kek.get_non_terminals_dict()

{'ADJ': 0,
 'ADV': 1,
 'DET': 2,
 'N': 3,
 'NN': 4,
 'NP': 5,
 'P': 7,
 'PP': 8,
 'S': 9,
 'T_NP': 6,
 'V': 11,
 'VP': 10}

In [4]:
kek.get_non_terminals_dict()

{'ADJ': 1,
 'ADV': 3,
 'Det': 8,
 'EPS': 11,
 'N': 2,
 'NN': 9,
 'NP': 5,
 'P': 10,
 'PP': 12,
 'S': 4,
 'TEMP': 0,
 'V': 7,
 'VP': 6}

Время проверить нашу грамматику.

In [5]:
correct_sentences = [
    'Mary eats a fish',
    'John eats a very very very tasty fish with a fork',
    'a dog eats a boy',
    'Mary eats John with a fork',
    'John eats a fork',
    'a very fish eats Mary'
]

not_correct_sentences = [
    'John',
    'Mary eats a eats',
    'boy eats dog',
    'John eats a very very very tasty fish with fork',
    'a Mary fork',
    'eats',
    'a'
]

parser = CYKParser(RULES)
print('Positve examples:')
print('-------------------------------------------------------------')
for sentence in correct_sentences:
    if parser.parse(sentence.split()):
        print('OK!', sentence)
    else:
        print('ERROR!', sentence)
print('*************************************************************')
print('Negative examples:')  
print('-------------------------------------------------------------')
for sentence in not_correct_sentences:
    if not parser.parse(sentence.split()):
        print('OK!', sentence)
    else:
        print('ERROR!', sentence)
print('*************************************************************')

Positve examples:
-------------------------------------------------------------
OK! Mary eats a fish
OK! John eats a very very very tasty fish with a fork
OK! a dog eats a boy
OK! Mary eats John with a fork
OK! John eats a fork
OK! a very fish eats Mary
*************************************************************
Negative examples:
-------------------------------------------------------------
OK! John
OK! Mary eats a eats
OK! boy eats dog
OK! John eats a very very very tasty fish with fork
OK! a Mary fork
OK! eats
OK! a
*************************************************************


## 3. Extended CYK Parser 

Модифицируйте парсер так, чтобы он возвращал цепочку правил, составляющих разбор.

In [49]:
from itertools import product
from collections import defaultdict

class CYKParser:
    def __init__(self, rules):
        self.rules = rules
    
    
    def get_non_terminals_dict(self):
        non_terminals_dict = {}
        i = 0
        for start in list(sorted(self.rules.keys())):
            if start not in non_terminals_dict:
                non_terminals_dict[start] = i
                i += 1
            for end in self.rules[start]:
                parts = end.split(' ')
                if len(parts) == 2:
                    for part in parts:
                        if part not in non_terminals_dict:
                            non_terminals_dict[part] = i
                            i += 1
        return non_terminals_dict
    
    
    def convert_rule_to_text(self, rule):
        return rule[0] + ' -> ' + ' '.join(rule[1:])
    
    
    def get_chain(self, r_mem, k_mem, i, j, nt_id):
        r = r_mem[i-1][j-1][nt_id]
        if len(r) == 2:
            return [self.convert_rule_to_text(r)]
        else:
            k = k_mem[i-1][j-1][nt_id]
            return [self.convert_rule_to_text(r)] + \
                    self.get_chain(r_mem, k_mem, k, j, self.get_non_terminals_dict()[r[1]]) + \
                    self.get_chain(r_mem, k_mem, i-k, j+k, self.get_non_terminals_dict()[r[2]])
    
    
    def parse(self, in_tokens):
        ntd = self.get_non_terminals_dict()
        p = []
        r_mem = []
        k_mem = []
        for i in range(len(in_tokens)):
            p.append([])
            r_mem.append([])
            k_mem.append([])
            for j in range(len(in_tokens)):
                p[i].append([])
                r_mem[i].append([])
                k_mem[i].append([])
                for k in range(len(ntd)):
                    p[i][j].append(False)
                    r_mem[i][j].append([])
                    k_mem[i][j].append([])
        for i in range(len(in_tokens)):
            word = in_tokens[i]
            for start in self.rules.keys():
                for end in self.rules[start]:
                    if len(end.split(' ')) == 1:
                        if end == word:
                            p[0][i][ntd[start]] = True
                            r_mem[0][i][ntd[start]] = [start, word]
        for i in range(2, len(in_tokens) + 1):
            for j in range(1, len(in_tokens) - i + 2):
                for k in range(1, i):
                    for start in self.rules.keys():
                        for end in self.rules[start]:
                            parts = end.split(' ')
                            if len(parts) == 2:
                                if p[k-1][j-1][ntd[parts[0]]] and p[i - k - 1][j + k - 1][ntd[parts[1]]]:
                                    p[i-1][j-1][ntd[start]] = True
                                    r_mem[i-1][j-1][ntd[start]] = [start] + parts
                                    k_mem[i-1][j-1][ntd[start]] = k
        if p[len(in_tokens) - 1][0][ntd['S']]:
            return [self.get_chain(r_mem, k_mem, len(in_tokens), 1, ntd['S'])]
        else:
            return []

In [47]:
CYKParser(RULES).parse('John eats a very very very tasty fish with a fork'.split(' '))

11 1 9
1 1 5
10 2 10
7 2 10
1 2 11
6 3 5
1 3 2
5 4 6
4 4 0
1 4 1
3 5 0
1 5 1
2 6 0
1 6 1
1 7 0
1 8 3
3 9 8
1 9 7
2 10 5
1 10 2
1 11 3


[['S -> NP VP',
  'NP -> John',
  'VP -> VP PP',
  'VP -> V NP',
  'V -> eats',
  'NP -> DET T_NP',
  'DET -> a',
  'T_NP -> ADJ N',
  'ADJ -> ADV ADJ',
  'ADV -> very',
  'ADJ -> ADV ADJ',
  'ADV -> very',
  'ADJ -> ADV ADJ',
  'ADV -> very',
  'ADJ -> tasty',
  'N -> fish',
  'PP -> P NP',
  'P -> with',
  'NP -> DET N',
  'DET -> a',
  'N -> fork']]

In [50]:
for sent in correct_sentences:
    for chain in CYKParser(RULES).parse(sent.split()):
        for rule in chain:
            print(rule)
        print()

S -> NP VP
NP -> Mary
VP -> V NP
V -> eats
NP -> DET N
DET -> a
N -> fish

S -> NP VP
NP -> John
VP -> VP PP
VP -> V NP
V -> eats
NP -> DET T_NP
DET -> a
T_NP -> ADJ N
ADJ -> ADV ADJ
ADV -> very
ADJ -> ADV ADJ
ADV -> very
ADJ -> ADV ADJ
ADV -> very
ADJ -> tasty
N -> fish
PP -> P NP
P -> with
NP -> DET N
DET -> a
N -> fork

S -> NP VP
NP -> DET N
DET -> a
N -> dog
VP -> V NP
V -> eats
NP -> DET N
DET -> a
N -> boy

S -> NP VP
NP -> Mary
VP -> VP PP
VP -> V NP
V -> eats
NP -> John
PP -> P NP
P -> with
NP -> DET N
DET -> a
N -> fork

S -> NP VP
NP -> John
VP -> V NP
V -> eats
NP -> DET N
DET -> a
N -> fork

S -> NP VP
NP -> DET T_NP
DET -> a
T_NP -> ADV N
ADV -> very
N -> fish
VP -> V NP
V -> eats
NP -> Mary



## Вместо заключения 

Вспомним про nltk и Earley парсер. Грамматики тут задаются следующим образом:

In [ ]:
import nltk
grammar = nltk.CFG.fromstring("""
    S -> NP VP
    NP -> Det ADJ N | NN
    VP -> V NP | VP PP | V
    Det -> 'a'
    ADJ -> 'tasty' | ADV ADJ | 
    ADV -> 'very'
    N -> 'fish' | 'fork' | 'dog' | 'boy'
    NN -> 'Mary' | 'John'
    V -> 'eats'
    PP -> P NP | 
    P -> 'with'
""")

Создадим парсер и применим к предложению.

In [ ]:
parser = nltk.EarleyChartParser(grammar)
trees = list(parser.parse('a very fish eats Mary'.split()))
for tree in trees:
    print(tree)

Чтобы увидеть сам разбор, достаточно установить параметр *trace* у парсера (по умолчанию trace=0).

In [ ]:
parser = nltk.EarleyChartParser(grammar, trace=1)
trees = parser.parse('Mary eats a fish'.split())
for tree in trees:
    print(tree)

In [ ]:
parser = nltk.EarleyChartParser(grammar, trace=2)
trees = parser.parse('John eats a very very very tasty fish with a fork'.split())
for tree in trees:
    print(tree)